# Extract Roles, Activities and Constraints from XML

In [9]:
import xml.etree.ElementTree as ET
import xmltodict

def getElements(xml):
    roles = []
    activities = {}
    rules = []
    activity_ids = {}
    
    with open(xml) as f:
        xml_string = f.read()    
    xml_dict = xmltodict.parse(xml_string)

    # add roles
    for event in xml_dict["dcr:definitions"]["dcr:dcrGraph"]["dcr:event"]:
        if isinstance(event, dict) and "@role" in event:
            if event["@role"] not in roles:
                roles.append(event["@role"])

    # add activities
    for event in xml_dict["dcr:definitions"]["dcr:dcrGraph"]["dcr:event"]:
        if isinstance(event, dict):
            if "@role" in event:
                activities[event["@description"]] = event["@role"]
                # helper for rules
                activity_ids[event["@id"]] = event["@description"]
            else:
                activities[event["@description"]] = None
                # helper for rules
                activity_ids[event["@id"]] = event["@description"]
        
    try:
        for relation in xml_dict["dcr:definitions"]["dcr:dcrGraph"]["dcr:relation"]:
            rules.append((relation["@type"],activity_ids[relation["@sourceRef"]],activity_ids[relation["@targetRef"]]))
    except Exception as e:
        print("no relation in model")

    return roles, activities, rules

# Find TP,TN,FP,FN for all Elements

In [ ]:
from sentence_transformers import SentenceTransformer
import numpy as np
model = SentenceTransformer("all-MiniLM-L6-v2")

def getScoreRoles(generated,gold):
    threshold = 0.7
    tp = 0
    tn = 0
    fp = 0
    fn = 0

    embeddings_generated = model.encode(generated)
    embeddings_gold = model.encode(gold)
    similarities = model.similarity(embeddings_gold, embeddings_generated)
    
    matrix = np.array(similarities)
    matrix_transposed = matrix.T

    # Has a role been correctly identified?
    for row in matrix_transposed:
        # Yes = true positive
        if any(threshold <= x for x in row):
            tp += 1
        # No = false positive
        else:
            fp += 1

    # Was the original role found?
    for row in matrix:
        # No = false negative
        if all(x <= threshold for x in row):
            fn += 1

    return  tp, tn, fp, fn

def getScoreActivities(generated,gold):
    threshold = 0.4
    tp = 0
    tn = 0
    fp = 0
    fn = 0

    generated_activities = []
    for key in generated:
        generated_activities.append(key)

    gold_activities = []
    for key in gold:
        gold_activities.append(key)

    embeddings_generated = model.encode(generated_activities)
    embeddings_gold = model.encode(gold_activities)
    similarities = model.similarity(embeddings_gold, embeddings_generated)
    
    matrix = np.array(similarities)
    matrix_transposed = matrix.T

    # Has a role been correctly identified?
    for row in matrix_transposed:
        # Yes = true positive
        if any(threshold <= x for x in row):
            tp += 1
        # No = false positive
        else:
            fp += 1

    # Was the original role found?
    for row in matrix:
        # No = false negative
        if all(x < threshold for x in row):
            fn += 1
    
    return  tp, tn, fp, fn


def getScoreRolesActivities(generated,gold):
    threshold_activites = 0.4
    threshold_roles = 0.7
    tp = 0
    tn = 0
    fp = 0
    fn = 0

    generated_activities = []
    generated_activity_roles = []
    for key in generated:
        generated_activities.append(key)
        generated_activity_roles.append(generated[key])

    gold_activities = []
    gold_activity_roles = []
    for key in gold:
        gold_activities.append(key)
        gold_activity_roles.append(gold[key])

    embeddings_generated = model.encode(generated_activities)
    embeddings_gold = model.encode(gold_activities)
    similarities = model.similarity(embeddings_gold, embeddings_generated)
    
    matrix = np.array(similarities)

    for row, values in enumerate(matrix):
        highest_value = 0
        highest_activity = ""
        for column, value in enumerate(values):
            # Which generated activity matches the gold activity best?
            if threshold_activites <= value and highest_value < value:
                highest_value = value
                highest_activity = generated_activities[column]
        if highest_value != 0:
            generated_role = generated[highest_activity]
            gold_role = gold[gold_activities[row]]
            if gold_role != None and generated_role != None:
                embedding_generated = model.encode(generated_role)
                embedding_gold = model.encode(gold_role)
                similarities = model.similarity(embedding_gold, embedding_generated)
                if threshold_roles <= similarities[0][0]:
                    tp += 1
                else:
                    fp += 1
            elif gold_role == None and generated_role == None:
                tp += 1
            else:
                fp += 1
        else:
            fn += 1 #activity was not correctly identified, therefore correct activity-role assignment not found either
    
    return  tp, tn, fp, fn

def getScoreRules(generated,gold):
    threshold_activity = 0.4
    threshold_constraint = 0.95
    tp = 0
    tn = 0
    fp = 0
    fn = 0

    generated_constraint = []
    generated_source = []
    generated_target = []

    for costraint in generated:
        generated_constraint.append(costraint[0])
        generated_source.append(costraint[1])
        generated_target.append(costraint[2])

    if len(generated_constraint) == 0:
        return 0, 0, 0, len(gold)

    gold_constraint = []
    gold_source = []
    gold_target = []
    for costraint in gold:
        gold_constraint.append(costraint[0])
        gold_source.append(costraint[1])
        gold_target.append(costraint[2])

    embed_gen_constraints = model.encode(generated_constraint)
    embed_gold_constraints = model.encode(gold_constraint)
    similarities_constraints = model.similarity(embed_gold_constraints, embed_gen_constraints)
    matrix_constraints = np.array(similarities_constraints)

    embed_gen_source = model.encode(generated_source)
    embed_gold_source = model.encode(gold_source)
    similarities_source = model.similarity(embed_gold_source, embed_gen_source)
    matrix_source = np.array(similarities_source)

    embed_gen_target = model.encode(generated_target)
    embed_gold_target = model.encode(gold_target)
    similarities_target = model.similarity(embed_gold_target, embed_gen_target)
    matrix_target = np.array(similarities_target)

    scores = [["x" for x in range(len(generated_constraint))] for y in range(len(gold_constraint))] 

    for row, constraints in enumerate(matrix_constraints):
        for column, value in enumerate(constraints):
            if threshold_constraint <= value and threshold_activity <= matrix_source[row][column] and threshold_activity <= matrix_target[row][column]:
                scores[row][column] = "tp" #constraint, source and target were correctly identified - true positive
            else:
                scores[row][column] = "fp"

    
    scores_transposed = np.array(scores).transpose()

    for row in scores_transposed:
        # did a generated constraint receive a match?
        if any(x == "tp" for x in row):
            tp += 1
        elif all(x == "fp" for x in row):
            fp += 1

    for row in scores:
        # the constraint was not found
        if all(x == "fp" for x in row):
            fn += 1
    
    return  tp, tn, fp, fn

# Calculate Scores

In [11]:
def calculateScores(true_positive,true_negative,false_positive,false_negative):
    # precision
    if 0 == (false_positive + true_positive):
        precision = 0
    else:
        precision = true_positive / ( false_positive + true_positive )
    # recall
    if 0 == (false_negative + true_positive):
        recall = 0
    else:
        recall = true_positive / ( false_negative + true_positive )
    # accuracy
    if 0 == (true_positive + false_negative + true_negative + false_positive):
        accuracy = 0
    else:
        accuracy = ( true_positive + true_negative ) / ( true_positive + false_negative + true_negative + false_positive)
    # jaccard
    if 0 == (true_positive + false_negative + false_positive):
        jaccard = 0
    else:
        jaccard = ( true_positive ) / ( true_positive + false_negative + false_positive)      
    # f1
    if 0 == (precision + recall):
        f1 = 0
    else:
        f1 = ( 2 * precision * recall ) / ( precision + recall )
        
    return f"{precision:.4f}",f"{recall:.4f}",f"{accuracy:.4f}",f"{jaccard:.4f}",f"{f1:.4f}"

# Run Analysis

In [12]:
import os
from pathlib import Path

result_analysis_path = './5_Test_Results'
result_xml_path = './4_Generated_XML'
gold_standard_path = './2_Gold_Standard'

for model_dir in os.listdir(result_xml_path):
    model_path = os.path.join(result_xml_path,model_dir)
    for pipeline_dir in os.listdir(model_path):
        pipeline_path = os.path.join(model_path,pipeline_dir)
        for prompt_dir in os.listdir(pipeline_path):
            prompt_path = os.path.join(pipeline_path,prompt_dir)

            true_positive_Roles = 0
            true_negative_Roles = 0
            false_positive_Roles = 0
            false_negative_Roles = 0
            true_positive_Activities = 0
            true_negative_Activities = 0
            false_positive_Activities = 0
            false_negative_Activities = 0
            true_positive_RolesActivities = 0
            true_negative_RolesActivities = 0
            false_positive_RolesActivities = 0
            false_negative_RolesActivities = 0
            true_positive_Rules = 0
            true_negative_Rules = 0
            false_positive_Rules = 0
            false_negative_Rules = 0

            num_generated_roles = 0
            num_generated_activities = 0
            num_generated_rules = 0
            num_gold_roles = 0
            num_gold_activities = 0
            num_gold_rules = 0


            for xml_file in os.listdir(prompt_path):
                generated_path = os.path.join(prompt_path,xml_file)

                process_name = xml_file.split(f'_{model_dir}')[0]
                process_name = process_name + ".xml"
                gold_path = os.path.join(gold_standard_path,process_name)

                if os.path.isfile(gold_path):
                    print(generated_path)
                    generated_roles, generated_activities, generated_rules = getElements(generated_path)
                    gold_roles, gold_activities, gold_rules = getElements(gold_path)

                    num_generated_roles += len(generated_roles)
                    num_generated_activities += len(generated_activities)
                    num_generated_rules += len(generated_rules)
                    num_gold_roles += len(gold_roles)
                    num_gold_activities += len(gold_activities)
                    num_gold_rules += len(gold_rules)
                    
                    if len(generated_roles) != 0:
                        tp, tn, fp, fn = getScoreRoles(generated_roles,gold_roles)
                        true_positive_Roles += tp
                        true_negative_Roles += tn
                        false_positive_Roles += fp 
                        false_negative_Roles += fn

                    if len(generated_activities) != 0:
                        tp, tn, fp, fn = getScoreActivities(generated_activities,gold_activities)
                        true_positive_Activities += tp
                        true_negative_Activities += tn
                        false_positive_Activities += fp 
                        false_negative_Activities += fn

                    if len(generated_activities) != 0:
                        tp, tn, fp, fn = getScoreRolesActivities(generated_activities,gold_activities)
                        true_positive_RolesActivities += tp
                        true_negative_RolesActivities += tn
                        false_positive_RolesActivities += fp 
                        false_negative_RolesActivities += fn

                    if len(generated_rules) != 0:
                        tp, tn, fp, fn = getScoreRules(generated_rules,gold_rules)
                        true_positive_Rules += tp
                        true_negative_Rules += tn
                        false_positive_Rules += fp 
                        false_negative_Rules += fn

            folder = Path(result_analysis_path) / model_dir
            folder.mkdir(parents=True, exist_ok=True) 
            file_name = f"{prompt_dir}.txt"
            file_path = folder / file_name 
            if os.path.exists(file_path):
                os.remove(file_path)


            p,r,a,j,f = calculateScores(true_positive_Roles,true_negative_Roles,false_positive_Roles,false_negative_Roles)
            with open(file_path, 'a+') as file:
                file.write(f"Numbers - Generated: Roles={num_generated_roles}, Activities={num_generated_activities}, Rules={num_generated_rules}\n")
                file.write(f"Numbers - Gold: Roles={num_gold_roles}, Activities={num_gold_activities}, Rules={num_gold_rules}\n\n")
                file.write("Roles\n")
                file.write(f"TP: {true_positive_Roles}, FP: {false_positive_Roles}, TN: {true_negative_Roles}, FN: {false_negative_Roles} \n")
                file.write(f"precision:{p}\n")
                file.write(f"recall: {r}\n")
                file.write(f"accuracy: {a}\n")
                file.write(f"jaccard: {j}\n")
                file.write(f"f1: {f}\n\n")

            p,r,a,j,f = calculateScores(true_positive_Activities,true_negative_Activities,false_positive_Activities,false_negative_Activities)
            with open(file_path, 'a+') as file:
                file.write(f"Activities\n")
                file.write(f"TP: {true_positive_Activities}, FP: {false_positive_Activities}, TN: {true_negative_Activities}, FN: {false_negative_Activities} \n")
                file.write(f"precision:{p}\n")
                file.write(f"recall: {r}\n")
                file.write(f"accuracy: {a}\n")
                file.write(f"jaccard: {j}\n")
                file.write(f"f1: {f}\n\n")

            p,r,a,j,f = calculateScores(true_positive_RolesActivities,true_negative_RolesActivities,false_positive_RolesActivities,false_negative_RolesActivities)
            with open(file_path, 'a+') as file:
                file.write(f"Activity-Role-Assignment\n")
                file.write(f"TP: {true_positive_RolesActivities}, FP: {false_positive_RolesActivities}, TN: {true_negative_RolesActivities}, FN: {false_negative_RolesActivities} \n")
                file.write(f"precision:{p}\n")
                file.write(f"recall: {r}\n")
                file.write(f"accuracy: {a}\n")
                file.write(f"jaccard: {j}\n")
                file.write(f"f1: {f}\n\n")

            p,r,a,j,f = calculateScores(true_positive_Rules,true_negative_Rules,false_positive_Rules,false_negative_Rules)
            with open(file_path, 'a+') as file:
                file.write(f"Constraints\n")
                file.write(f"TP: {true_positive_Rules}, FP: {false_positive_Rules}, TN: {true_negative_Rules}, FN: {false_negative_Rules} \n")
                file.write(f"precision:{p}\n")
                file.write(f"recall: {r}\n")
                file.write(f"accuracy: {a}\n")
                file.write(f"jaccard: {j}\n")
                file.write(f"f1: {f}\n")

./4_Generated_XML/llama3.2/Pipeline_A1/Pipeline_A1_Prompt_1/3_Computer_Repair_Service_llama3.2_Pipeline_A1_Prompt_1_2025-07-29_21:35:39.851852.xml
./4_Generated_XML/llama3.2/Pipeline_A1/Pipeline_A1_Prompt_1/4_Expense_Handling_Process_llama3.2_Pipeline_A1_Prompt_1_2025-07-30_17:01:12.572011.xml


/tmp/ipykernel_2111087/3205669642.py:16: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword arguments. To learn more, see the migration guide https://numpy.org/devdocs/numpy_2_0_migration_guide.html#adapting-to-changes-in-the-copy-keyword
  matrix = np.array(similarities)
/tmp/ipykernel_2111087/3205669642.py:55: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword arguments. To learn more, see the migration guide https://numpy.org/devdocs/numpy_2_0_migration_guide.html#adapting-to-changes-in-the-copy-keyword
  matrix = np.array(similarities)
/tmp/ipykernel_2111087/3205669642.py:100: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword arguments. To learn more, see the migrat

./4_Generated_XML/llama3.2/Pipeline_A1/Pipeline_A1_Prompt_1/8_Mikkeller_webshop_llama3.2_Pipeline_A1_Prompt_1_2025-07-30_17:07:46.627446.xml
./4_Generated_XML/llama3.2/Pipeline_A1/Pipeline_A1_Prompt_1/10_Design_approval_llama3.2_Pipeline_A1_Prompt_1_2025-07-30_17:24:54.931166.xml
./4_Generated_XML/llama3.2/Pipeline_A1/Pipeline_A1_Prompt_1/15_Long_term_renting_process_llama3.2_Pipeline_A1_Prompt_1_2025-07-30_17:44:05.648200.xml
./4_Generated_XML/llama3.2/Pipeline_A1/Pipeline_A1_Prompt_1/16_Bicycle_Rental_Process_llama3.2_Pipeline_A1_Prompt_1_2025-07-30_17:48:33.448621.xml
./4_Generated_XML/llama3.2/Pipeline_A1/Pipeline_A1_Prompt_1/18_nan_llama3.2_Pipeline_A1_Prompt_1_2025-07-30_17:56:02.544241.xml
./4_Generated_XML/llama3.2/Pipeline_A1/Pipeline_A1_Prompt_1/1_Hospital_Workflow_llama3.2_Pipeline_A1_Prompt_1_2025-07-30_17:59:07.119443.xml
./4_Generated_XML/llama3.2/Pipeline_A1/Pipeline_A1_Prompt_1/5_Quality_control_of_medical_products_llama3.2_Pipeline_A1_Prompt_1_2025-07-30_18:01:17.11522